In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../../data/rides/Taxi_Trips_Sampled_Cleaned.parquet')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171339 entries, 0 to 171338
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Trip Start Timestamp  171339 non-null  datetime64[ns]
 1   Trip End Timestamp    171339 non-null  datetime64[ns]
 2   Trip Seconds          171339 non-null  float64       
 3   Trip Miles            171339 non-null  float64       
 4   Pickup Census Tract   171339 non-null  int64         
 5   Dropoff Census Tract  171339 non-null  int64         
 6   Fare                  171339 non-null  float64       
 7   Tips                  171339 non-null  float64       
 8   Tolls                 171339 non-null  float64       
 9   Extras                171339 non-null  float64       
 10  Trip Total            171339 non-null  float64       
 11  Payment Type          171339 non-null  object        
 12  Company               171339 non-null  object        
 13 

In [5]:
df.describe()

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tips,Tolls,Extras,...,week_start,month_start,hour_end,4_hour_block_end,day_end,week_end,month_end,datetime,temp,Precip
count,171339,171339,171339.000000,171339.000000,1.713390e+05,1.713390e+05,171339.000000,171339.000000,171339.000000,171339.000000,...,171339.0,171339.000000,171339.000000,171339.000000,171339.000000,171339.0,171339.000000,171339,171339.000000,171339.000000
mean,2015-06-25 10:46:55.791501056,2015-06-25 11:01:00.375045632,779.412107,3.307421,1.703136e+10,1.703135e+10,11.411371,1.323660,0.000357,0.722346,...,25.967842,6.295403,14.051856,3.137208,15.611933,25.968361,6.295630,2015-06-25 10:47:03.140090880,9.336972,0.018957
min,2015-01-01 00:00:00,2015-01-01 00:00:00,0.000000,0.100000,1.703101e+10,1.703101e+10,0.000000,0.000000,0.000000,0.000000,...,1.0,1.000000,0.000000,0.000000,1.000000,1.0,1.000000,2015-01-01 00:00:00,-23.966000,0.000000
25%,2015-03-28 11:45:00,2015-03-28 11:45:00,360.000000,0.800000,1.703108e+10,1.703108e+10,5.850000,0.000000,0.000000,0.000000,...,13.0,3.000000,10.000000,2.000000,8.000000,13.0,3.000000,2015-03-28 12:00:00,1.801500,0.000000
50%,2015-06-19 04:30:00,2015-06-19 04:45:00,565.000000,1.400000,1.703128e+10,1.703124e+10,7.650000,0.000000,0.000000,0.000000,...,25.0,6.000000,15.000000,3.000000,16.000000,25.0,6.000000,2015-06-19 04:00:00,10.559000,0.000000
75%,2015-09-22 07:30:00,2015-09-22 08:00:00,900.000000,2.900000,1.703183e+10,1.703183e+10,11.250000,2.000000,0.000000,1.000000,...,39.0,9.000000,19.000000,4.000000,23.000000,39.0,9.000000,2015-09-22 08:00:00,17.721609,0.000000
max,2015-12-31 12:00:00,2015-12-31 12:00:00,5126.000000,29.400000,1.703198e+10,1.703198e+10,58.250000,50.000000,50.000000,51.500000,...,53.0,12.000000,23.000000,5.000000,31.000000,53.0,12.000000,2015-12-31 12:00:00,33.100000,1.000000
std,NaN,NaN,670.115217,4.816970,3.412321e+05,3.330212e+05,9.482748,2.172617,0.121572,1.037810,...,14.632981,3.367677,6.285732,1.541644,8.663581,14.632868,3.367632,NaN,11.538467,0.136372


In [6]:
df.columns

Index(['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Pickup Census Tract', 'Dropoff Census Tract', 'Fare',
       'Tips', 'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company',
       'hour_start', '4_hour_block_start', 'day_start', 'week_start',
       'month_start', 'hour_end', '4_hour_block_end', 'day_end', 'week_end',
       'month_end', 'h3_07_Pickup', 'h3_07_Dropoff', 'h3_08_Pickup',
       'h3_08_Dropoff', 'h3_09_Pickup', 'h3_09_Dropoff', 'Pickup Centroid',
       'Dropoff Centroid', 'datetime', 'temp', 'Precip'],
      dtype='object')

In [9]:
len(df["Pickup Census Tract"].unique())

267

In [10]:
len(df["Pickup Centroid"].unique())

267

In [14]:
len(df["h3_08_Pickup"].unique())

167

In [23]:
timeinterval_length = 1
trips_c = df.copy()

trips_c["datetime_start_floored"] = trips_c["Trip Start Timestamp"].dt.floor(
        f"{timeinterval_length}H"
)

trips_c.groupby(["datetime_start_floored", "h3_07_Pickup", "h3_07_Dropoff"]).size().to_frame("demand")

demand
datetime_start_floored h3_07_Pickup    h3_07_Dropoff          
2015-01-01 00:00:00    872664c10ffffff 872664c14ffffff       1
                                       872664c16ffffff       2
                       872664c11ffffff 872664c12ffffff       1
                                       872664c1affffff       1
                       872664c12ffffff 872664c13ffffff       1
...                                                        ...
2015-12-31 11:00:00    872664c1effffff 872664c1effffff       3
                                       872664cadffffff       1
                       872664ca9ffffff 872664c1effffff       1
2015-12-31 12:00:00    872664c1effffff 872664c1affffff       1
                       872664cadffffff 872664cadffffff       1

[105412 rows x 1 columns]